# Obtain the Data
The second step of the ROSEMED process is to obtain the data.

Here I will write the various scripts and functions that I used to obtain the data.

## Initial Data
The initial dataset was given to me by the Flatiron school, and is a well known data science set, used in many Kaggle competitions and various online tutorials.

I was given a set that had additional inconsistencies thrown in by the faculty at Flatiron, as a way to practice the techniques we have been learning.

In [2]:
import pandas as pd

In [231]:
# load in initial dataset
initial_df = pd.read_csv('../data/raw/kc_house_data.csv')

In [232]:
initial_df.head(3)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062


In [233]:
initial_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

## Additional Data
Now it is time to bring in additional data that I chose from the research phase.

The first steps are to transform the zipcode into something more useable. We will use a python library for that, called 'uszipcode'. We can use it to look up a zipcode and get important stats related to income or neighborhoods.

That means finding a way to get the Walk Scores, and the GreatSchools information I chose to use earlier.

### Convert Zipcodes

In [170]:
from uszipcode import SearchEngine, SimpleZipcode
search = SearchEngine(simple_zipcode=False)

In [234]:
def zco(x, attribute):
    '''Take zipcode and return selected attribute'''
    city = search.by_zipcode(x).to_dict()
    return city[attribute] if attribute in city else x

In [333]:
# decide which information to use
attributes = ['median_household_income', 'median_home_value']

In [334]:
zco(98003, 'median_home_value')

221600

In [335]:
for att in attributes:
    initial_df[att] = initial_df['zipcode'].apply(lambda x: zco(x, att))

In [336]:
initial_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       21597 non-null  int64  
 1   date                     21597 non-null  object 
 2   price                    21597 non-null  float64
 3   bedrooms                 21597 non-null  int64  
 4   bathrooms                21597 non-null  float64
 5   sqft_living              21597 non-null  int64  
 6   sqft_lot                 21597 non-null  int64  
 7   floors                   21597 non-null  float64
 8   waterfront               19221 non-null  float64
 9   view                     21534 non-null  float64
 10  condition                21597 non-null  int64  
 11  grade                    21597 non-null  int64  
 12  sqft_above               21597 non-null  int64  
 13  sqft_basement            21597 non-null  object 
 14  yr_built              

### Walk Score
This is the process I took to obtain the Walk Score for each house in the dataset

In [72]:
# required imports
import requests
import dotenv
import os
import geocoder
import requests

# use dotenv to keep API keys private
dotenv_path = os.path.join(os.pardir, '.env')
dotenv.load_dotenv(dotenv_path)

# start a session
s = requests.Session()

In [15]:
# read in main dataframe
df = pd.read_csv('../data/raw/kc_house_data.csv')

In [16]:
df.sample(3)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
8287,7313200120,10/31/2014,605000.0,4,3.25,2885,33671,2.0,0.0,0.0,...,8,2885,0.0,1984,NaN,98027,47.5174,-122.046,1910,16000
15459,6072400280,6/19/2014,619850.0,4,2.50,2270,9247,1.0,0.0,0.0,...,8,1500,770.0,1972,0.0,98006,47.5602,-122.176,2270,9163
16482,8838900167,5/9/2014,542500.0,4,2.50,2330,14289,2.0,0.0,0.0,...,8,2330,0.0,1978,0.0,98007,47.5916,-122.148,2210,12823


In [17]:
# get list of zipcodes, will use these to break up the api calls into 
# smaller chunks
zips = list(df['zipcode'].unique())

The Walk Score API requires that we send the street address with the lat and lon in our query, so first we have to use the Bing API, and save the street address in our DataFrame

In [24]:
def fetch_address(house):
    """Reverse lookup the address for a house using its lat and lon"""
    g = geocoder.bing([house.lat, house.long], method='reverse', key=key)

    print(g.current_result, end="\r", flush=True)
    return g.current_result

In [25]:
# create test house and test the fetch_address function
test_house = df.sample()
fetch_address(test_house)

[1251 22nd Ave E, Seattle, WA 98112]

In [ ]:
# main loop for looking up address, split up by zipcode
for code in zips:
    temp_df = df[df['zipcode'] == code].copy()
    temp_df['address'] = temp_df.apply(lambda x: fetch_address(x), axis=1)
    temp_df.to_csv(f'../data/interim/addressed_houses_{code}.csv')
    print(f'saved addresses in zip: {code}')

This will save a temporary DataFrame for each zipcode. Let's import them all and combine them into our new dataframe with addresses.

In [27]:
def combine_temp_frames(name='temp', directory='../data/interim/'):
    """Take the name of sub dataframes to read and combine
    
    Args: 
        name (str): contains str to check for when parsing file names
        directory (str): directory the temp frames are located in
        
    Returns:
        combined_df (DataFrame): new combined dataframe"""
    
    interim_paths = os.listdir(directory)
    
    combined_df = pd.DataFrame()
    for path in interim_paths:
        if name in path:
            temp_df = pd.read_csv(directory+path, index_col=0)
            combined_df = pd.concat([combined_df, temp_df], axis=0)
            
    return combined_df

In [28]:
# use our function to read in the dataframe that now has addresses
addressed_df = combine_temp_frames('addressed')

In [192]:
addressed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21597 entries, 65 to 21430
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long 

Now that we have the street address for each house, we can start looping through the data frame and looking up the Walk Scores for each house.

In [29]:
# we are going to need our API key for Walk Scores
ws_key = os.getenv("WS")

In [30]:
# we will need a function to run for each house, that will contact the API
# and return the desired information
import json
def fetch_scores(house):
    """Sends request to Walk Score API
    
    Args:
        house (DataFrame row): row representing a house, must have 'address', 'lat', 'long'
        
    Returns:
        (dict): contains walkscore, transit_score, bike_score, or {'error': 41} if our of api calls"""
    
    # url for api endpoint
    url = 'http://api.walkscore.com/score'
    
    # desired params
    params = {
    'format': 'json',
    'address': house['address'],
    'lat': house['lat'],
    'lon': house['long'],
    'transit': 1, 
    'bike': 1,
    'wsapikey': ws_key}
    
    # send request
    res = s.get(url, params=params)
    
    # turn response into json
    result = json.loads(res.content)
    
    if result['status'] != 1:
        
        # if unsuccesful:
        return {
            'error_code': result['status']
        }
    else:
        
        # if succesful:
        return {
            'walkscore': result['walkscore'] if 'walkscore' in result else None,
            'transit_score': result['transit']['score'] if 'transit' in result else None,
            'bike_score': result['bike']['score'] if 'bike' in result else None
        }

In [210]:
# test on one sample from df
test_house = addressed_df.sample()
print(fetch_scores(test_house))

{'error_code': 41}


In [398]:
len(zips)

70

In [384]:
print(finished_zips)

[98005, 98011, 98039, 98038, 98010, 98004, 98166, 98199, 98006, 98007, 98198, 98177, 98188, 98003, 98002, 98148, 98028, 98014, 98001, 98029, 98112, 98119, 98112, 98052, 98027, 98117, 98058, 98001, 98056, 98166, 98023, 98070, 98148, 98105, 98042, 98008, 98059, 98122, 98144, 98004, 98005, 98034, 98075, 98116, 98010, 98118, 98199, 98032, 98045, 98102, 98077, 98108, 98168, 98177, 98065, 98029, 98006, 98109, 98022, 98033, 98155, 98024, 98011, 98031, 98106, 98072, 98188, 98014, 98055, 98039, 98178, 98125]


In [385]:
len(finished_zips)

72

In [382]:
daily_limit = 5000
finished_zips = [98005, 98011, 98039, 98038, 98010, 98004, 98166, 98199, 98006, 98007, 98198, 98177, 98188, 98003, 98002, 98148, 98028, 98014, 98001, 98029, 98112, 98119, 98112, 98052, 98027, 98117, 98058, 98001, 98056, 98166, 98023, 98070, 98148, 98105,
                 98042, 98008, 98059, 98122, 98144, 98004, 98005, 98034, 98075, 98116, 98010, 98118, 98199, 98032, 98045, 98102, 98077, 98108, 98168, 98177, 98065, 98029, 98006, 98109, 98022, 98033, 98155, 98024, 98011, 98031, 98106, 98072, 98188, 98014, 98055, 98039]

In [409]:
missed_zips = [98040, 98136, 98126, 98146, 98103, 98019, 98092, 98053, 98133, 98030]

In [ ]:
# create a loop that will use all of the API calls granted in a one day period
while daily_limit > 0:
    for code in missed_zips:
        temp_df = addressed_df[addressed_df['zipcode'] == code].copy()
        
        if daily_limit - temp_df.shape[0] < 0:
            print(f'Out of calls. Last zip: {code}')
            daily_limit = 0
            break
        
        temp_df['scores'] = temp_df.apply(lambda x: fetch_scores(x), axis=1)
        temp_df.to_csv(f'../data/interim/scored_houses_{code}.csv')
        daily_limit -= temp_df.shape[0]
        finished_zips.append(code)
        print(f'saved addresses in zip: {code}')

Now we have a sub DataFrame for each zipcode in King County. Let's combine them all into one and save it.

In [411]:
scored_df = combine_temp_frames('scored_houses')

In [412]:
scored_df.drop_duplicates(inplace=True)

In [413]:
scored_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21597 entries, 46 to 21539
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long 

In [414]:
scored_df['walk_score'] = scored_df['scores'].apply(lambda x: x.split(',')[0].split(':')[1])
scored_df['transit_score'] = scored_df['scores'].apply(lambda x: x.split(',')[1].split(':')[1])
scored_df['bike_score'] = scored_df['scores'].apply(lambda x: x.split(',')[2].split(':')[1].strip('}'))

In [415]:
scored_df = scored_df[['id', 'walk_score', 'transit_score', 'bike_score']].copy()
scored_df = initial_df.set_index('id').join(scored_df.set_index('id'))
scored_df.to_csv('../data/interim/scored_df.csv')

In [416]:
scored_df = pd.read_csv('../data/interim/scored_df.csv', index_col=0)

In [417]:
(scored_df['walk_score'] == ' None').sum()

1320

In [418]:
list(scored_df[scored_df['walk_score'].isna()]['zipcode'].unique())

[]

### Great Schools
This is the process I took to obtain the Great Schools distances for each house in the dataset

In [75]:
# grab our geo code API key
geo_key = os.getenv("GEO_CODE")
bing_key = os.getenv("BING")

Define some helper functions

In [41]:
from haversine import haversine

def find_nearest(source, targets_df, num):
    """Use haversine to calculate closes schools to target house
    
    Args: 
        source (DataFrame row): source, must have 'lat' and 'lon'
        targets_df (DataFrame): DF of targets, each must have 'lat' and 'lon'
        num (int): num of nearest targets to return
        
    Returns:
        results_df (DataFrame): DataFrame of 'num' closest target rows
    """
    
    results_df = schools_df.copy()
    results_df['distance_to_house'] = results_df.apply(lambda x: haversine((source['lat'], source['long']), (x['lat'], x['long'])), axis='columns')
    return results_df.sort_values('distance_to_house').head(num)

In [83]:
def school_commutes(house, schools_df, num_schools=10):
    """Return the driving time and distance from house to 'num_schools' schools
    
    Args: 
        house (DataFrame row): house to find distances for
        schools_df (DataFrame): dataframe to find school lat/lon
        num_schools (int): how many schools to check distance
        
    Returns:
        results (list): list of 'num_schools' results, drive time (mins) and drive distance (miles)
    """
    bing_url = f'https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?key={bing_key}'
    closest_schools_df = find_nearest(house, schools_df, num_schools)
    
    # build params
    school_coords = zip(round(closest_schools_df['lat'], 6), round(closest_schools_df['long'], 6))
    params = { 
        'origins': [
            { 'latitude': house['lat'], 'longitude': house['long'] }
        ], 
        'destinations': [], 
        'travelMode': 'driving'}
    
    for coords in school_coords:
        destination = { 'latitude': coords[0], 'longitude': coords[1] }
        params['destinations'].append(destination)
    
    # build result row
    results = []
    # return result row
    res = requests.post(bing_url, json=params)
    content = json.loads(res.content)
    return content['resourceSets'][0]['resources'][0]['results']

In [43]:
def closest_school_score(distance_results):
    """Take some distance results and find nearest result"""
    
    distance = 1000000 # max
    duration = 1000000 # max
    for result in distance_results:
        if result['travelDistance'] < distance:
            distance = result['travelDistance']
        if result['travelDuration'] < duration:
            duration = result['travelDuration']
    return { 'distance': distance, 'duration': duration }

In [44]:
def save_closest_school(house, schools_df, num_schools=10):
    results = school_commutes(house, schools_df, num_schools)
    return closest_school_score(results)

The 'schools_df' used in these functions was created using a python script in '../src/data/SchoolCrawler.py'

In [84]:
# do a test
from pprint import pprint
schools_df = pd.read_csv('../data/raw/school_ratings.csv', index_col=0)
results = school_commutes(addressed_df.iloc[1], schools_df, 10)
pprint(results)

[{'destinationIndex': 0,
  'originIndex': 0,
  'totalWalkDuration': 0,
  'travelDistance': 8.649,
  'travelDuration': 12.15},
 {'destinationIndex': 1,
  'originIndex': 0,
  'totalWalkDuration': 0,
  'travelDistance': 8.828,
  'travelDuration': 12.367},
 {'destinationIndex': 2,
  'originIndex': 0,
  'totalWalkDuration': 0,
  'travelDistance': 17.797,
  'travelDuration': 22.717},
 {'destinationIndex': 3,
  'originIndex': 0,
  'totalWalkDuration': 0,
  'travelDistance': 9.534,
  'travelDuration': 15.117},
 {'destinationIndex': 4,
  'originIndex': 0,
  'totalWalkDuration': 0,
  'travelDistance': 10.982,
  'travelDuration': 14.617},
 {'destinationIndex': 5,
  'originIndex': 0,
  'totalWalkDuration': 0,
  'travelDistance': 9.129,
  'travelDuration': 21.433},
 {'destinationIndex': 6,
  'originIndex': 0,
  'totalWalkDuration': 0,
  'travelDistance': 10.965,
  'travelDuration': 20.033},
 {'destinationIndex': 7,
  'originIndex': 0,
  'totalWalkDuration': 0,
  'travelDistance': 13.673,
  'travelD

In [85]:
# run a small subset to test out the loop
sample_df = df.sample(25).copy()
sample_df['top_schools'] = sample_df.apply(lambda x: save_closest_school(x, schools_df), axis=1)

In [89]:
sample_df['top_schools'].sample(3)

9562     {'distance': 11.212, 'duration': 19.45}
4576        {'distance': 2.816, 'duration': 3.8}
13214     {'distance': 8.895, 'duration': 12.25}
Name: top_schools, dtype: object

In [ ]:
# now  that we know it works, run it for the entire dataset,
# breaking it down into zipcode sized chunks
for z in zips:
    temp_df = df[df['zipcode'] == z].copy()
    temp_df['top_schools'] = temp_df.apply(lambda x: save_closest_school(x, schools_df), axis=1)
    temp_df.to_csv(f'../data/interim/temp_{z}.csv')
    print(f'completed homes in zip {z}')

Now we can recombine the temp sub dataframes, create a column for each metric, and have our completed dataframe

In [342]:
schools_df = combine_temp_frames('temp')

In [343]:
schools_df['school_kms'] = schools_df['top_schools'].map(lambda x: float(x.split(' ')[1].strip(',')))
schools_df['school_mins'] = schools_df['top_schools'].map(lambda x: float(x.split(' ')[3].strip(',}')))

In [344]:
schools_df.sample(3)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,geometry,top_schools,school_kms,school_mins
12083,1432900310,4/24/2015,320000.0,4,1.5,2020,8474,1.0,0.0,0.0,...,0.0,98058,47.4579,-122.170,1720,8166,POINT (-122.17 47.4579),"{'distance': 18.525, 'duration': 19.483}",18.525,19.483
16355,4083301950,11/20/2014,580000.0,2,1.0,1040,3200,1.0,0.0,0.0,...,0.0,98103,47.6558,-122.334,1890,4000,POINT (-122.334 47.6558),"{'distance': 4.057, 'duration': 7.75}",4.057,7.750
13513,1787600209,9/9/2014,432500.0,3,2.5,1340,8867,2.0,0.0,0.0,...,0.0,98125,47.7240,-122.327,1630,7287,POINT (-122.327 47.724),"{'distance': 5.975, 'duration': 7.7}",5.975,7.700


In [345]:
schools_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21597 entries, 70 to 21396
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long 

In [346]:
schools_df = schools_df[['school_kms', 'school_mins', 'id']].copy()

In [419]:
scored_df = pd.read_csv('../data/interim/scored_df.csv', index_col=0)
combined_df = schools_df.set_index('id').join(scored_df)

In [420]:
combined_df = combined_df.reset_index()

In [421]:
combined_df.drop_duplicates(inplace=True)

In [422]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21597 entries, 0 to 22670
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       21597 non-null  int64  
 1   school_kms               21597 non-null  float64
 2   school_mins              21597 non-null  float64
 3   date                     21597 non-null  object 
 4   price                    21597 non-null  float64
 5   bedrooms                 21597 non-null  int64  
 6   bathrooms                21597 non-null  float64
 7   sqft_living              21597 non-null  int64  
 8   sqft_lot                 21597 non-null  int64  
 9   floors                   21597 non-null  float64
 10  waterfront               19221 non-null  float64
 11  view                     21534 non-null  float64
 12  condition                21597 non-null  int64  
 13  grade                    21597 non-null  int64  
 14  sqft_above            

In [423]:
combined_df.to_csv('../data/interim/combined_df.csv')